In [6]:
import sqlite3
import random
import shutil
import numpy as np
import pandas as pd

In [7]:
def demoValue(col_type, min_val, max_value, demo_string_data):
    value = ""
    
    if col_type == 'integer':
        value = random.randint(min_val, max_value)
        value = value
    else:
        demo_data_index = random.randint(0, len(demo_string_data.index) - 1)
        value = demo_string_data[col_type][demo_data_index].replace("'", " ")
    return str(value)

In [8]:
def createDBSchema(max_num_of_tables, min_num_of_cols, max_num_of_cols, demo_string_data):
    db_schema = []
    num_table = 0
    num_of_tables = random.randint(1, max_num_of_tables)
    
    possible_col_types = ['integer']
    for key in demo_string_data.keys():
        possible_col_types.append(key)
    
    while num_table < num_of_tables:
        
        table = []
        
        table_name = "tab_" + str(num_table)
        table.append(table_name)
        
        cols = []
        num_of_cols = random.randint(min_num_of_cols, max_num_of_cols)
        num_col = 0
        while num_col < num_of_cols:
            col = []
            col_type = random.randint(0, 1)
            col.append("col" + str(num_col))
            # col type is number
            if col_type == 0:
                col.append(possible_col_types[col_type])
            # col type is text
            else:
                col_text_type = random.randint(1, len(possible_col_types) - 1)
                col.append(possible_col_types[col_text_type])
            cols.append(col)
            num_col += 1
        
        table.append(cols)       
        db_schema.append(table)
        num_table += 1
    return db_schema

In [9]:
def insertSchema(db_schema, cur):
    
    for table in db_schema:
        table_name = table[0]
        cols = table[1]
        
        col_string = ""
        for col in cols:
            col_string += col[0] + " " + col[1] + ", "
        col_string = col_string[:-2]
        
        table_query = "CREATE TABLE IF NOT EXISTS " + table_name + " (id integer PRIMARY KEY, " + col_string + ")"
        cur.execute(table_query)

In [10]:
def insertData(db_schema, cur, min_num_of_values, max_num_of_values, min_int_val, max_int_val, demo_string_data):
    
    for table in db_schema:
        
        table_name = table[0]
        cols = table[1]
        
        row = 0
        row_insert = ""
        same_row = 0
        num_of_values = random.randint(min_num_of_values, max_num_of_values)
        
        while row < num_of_values:
            if same_row != 1:
                values = ""
                col_string = ""
                for col in cols:
                    col_string += col[0] + ", "
                    values += "'" + demoValue(col[1], min_int_val, max_int_val, demo_string_data) + "', "
                
                row_insert = "INSERT INTO " + table_name + " (" + col_string[:-2] + ") VALUES ( " + values[:-2] + ")"
                cur.execute(row_insert)
            row += 1

In [11]:
db_num = 100
db_version = 25

# load demo database
conn = sqlite3.connect("Demo_Data_DB.sqlite")
query = "SELECT * FROM DEMO_DATA;"
demo_string_data = pd.read_sql_query(query, conn)
conn.close()

for num in range(100):
    
    num_of_versions = 4
    
    min_int_val = -1 * (num + 1)*1000
    max_int_val = (num + 1)*1000
    
    min_num_of_values = (num)*100 + 1
    max_num_of_values = (num)*100 + 50
    
    for version in range(num_of_versions):
        if version == 0:
            db_name = "db_" + str(num) + "_v" + str(version) + ".sqlite"
            conn = sqlite3.connect(db_name)
            cur = conn.cursor()
            
            max_num_of_tables = 10 - num % 10
            min_num_of_cols = num % 10 + 1
            max_num_of_cols = num % 10 + 3
            
            db_schema = createDBSchema(max_num_of_tables, min_num_of_cols, max_num_of_cols, demo_string_data.iloc[: , 1:])
            insertSchema(db_schema, cur)
            insertData(db_schema, cur, min_num_of_values, max_num_of_values, min_int_val, max_int_val, demo_string_data.iloc[: , 1:])
            conn.commit()
            conn.close()
        else:
            db_name = "db_" + str(num) + "_v" + str(version) + ".sqlite"
            conn = sqlite3.connect(db_name)
            cur = conn.cursor()
            insertData(db_schema, cur, 1, 10, min_int_val, max_int_val, demo_string_data.iloc[: , 1:])
            conn.commit()
            conn.close()
            
        if version < num_of_versions - 1:
            db_next_name = "db_" + str(num) + "_v" + str(version + 1) + ".sqlite"
            shutil.copy(db_name, db_next_name)
        path = "demo_dbs/"
        shutil.move(db_name, path + db_name)